In [46]:
params = (a, b, c)

In [11]:
params

(4, 5, 6)

In [12]:
values = (7, 8, 9)

In [13]:
params = values

In [14]:
params

(7, 8, 9)

In [15]:
a

4

In [22]:
*params = values

SyntaxError: starred assignment target must be in a list or tuple (<ipython-input-22-4133f5399992>, line 1)

In [18]:
values

(7, 8, 9)

In [23]:
params = ('a', 'b', 'c')

In [26]:
*params,  = values

In [27]:
params

[7, 8, 9]

In [28]:
b

5

In [29]:
values

(7, 8, 9)

In [31]:
*params,

(7, 8, 9)

SyntaxError: invalid syntax (<ipython-input-33-d4248c49f934>, line 1)

In [34]:
*a,b,c  = values

In [35]:
b

8

In [36]:
c

9

In [47]:
values = 1,2,3

In [48]:
values

(1, 2, 3)

In [49]:
a,b,c = values

In [50]:
b

2

In [37]:
import pickle

In [41]:
with open('xxx.pickle', 'wb') as file:
    pickle.dump(values, file)

In [43]:
with open('xxx.pickle', 'rb') as f:
    a = pickle.load(f)

In [44]:
a

(7, 8, 9)